In [ ]:
!python3 -m pip install --upgrade paddlenlp==2.5.2

Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/, https://pypi.tuna.tsinghua.edu.cn/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 42.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.7/512.7 kB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/194.6 kB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 38.7 MB/s eta 0:00:00
  Attempting uninstall: paddlefsl
    Found existing installation: paddlefsl 1.0.0
    Uninstalling paddlefsl-1.0.0:
      Successfully uninstalled paddlefsl-1.0.0
  Attempting uninstall: click
    Found existing installation: Click 7.0
    Uninstalling Click-7.0:
      Successfully uninstalled Click-7.0
  Attempting uninstall: paddlenlp
    Found existing instal

In [ ]:
!git clone https://github.com/PaddlePaddle/PaddleNLP.git

In [ ]:
!python3 -m  pip install scikit-learn==1.0.2

Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/, https://pypi.tuna.tsinghua.edu.cn/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 13.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2

[notice] A new release of pip available: 22.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
from functools import partial
import argparse
import os
import random
import time
import re
import json
import codecs
import pandas as pd
import numpy as np

import numpy as np
import pandas as pd
import paddle
import paddle.nn.functional as F
import paddle.nn as nn
from paddle.metric import Metric
from tqdm import tqdm

import paddlenlp as ppnlp
from paddlenlp.data import Stack, Tuple, Pad
from paddlenlp.datasets import load_dataset
from paddlenlp.transformers import LinearDecayWithWarmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score

In [ ]:
train_path='work/Business_Industry_URLS_wText.csv'
train_df = pd.read_csv(train_path)
train_df = train_df[['Business_Name','Industry_Group_Description', 'Text']]
train_df = train_df.dropna(subset=['Text'])

In [ ]:
len(train_df)

5841

# **Data Preprocessing**

In [ ]:
train_df = train_df.dropna(subset=['Text'],axis=0)
train_df['Text'] = train_df['Business_Name'] + ':' + train_df['Text']
train_df = train_df[['Business_Name', 'Industry_Group_Description', 'Text']]
train_df['Industry_Group_Description'] = train_df['Industry_Group_Description'].str.replace(',', ' ', regex=False)

In [ ]:
# train_df['Text'] = train_df['Business_Name'] + ': ' + train_df['Text']

In [ ]:
len(train_df)

5841

In [ ]:
train_df.head()

,Business_Name,Industry_Group_Description,Text
0,Mariposa Hair Salon,Personal Care Services,Mariposa Hair Salon:When you choose Mariposa a...
1,Mountain Dew Behavior Health,Other Ambulatory Health Care Services,Mountain Dew Behavior Health:Mountain Dew Beha...
2,Johnson Brothers Nevada Inc,Beer Wine and Distilled Alcoholic Beverage M...,Johnson Brothers Nevada Inc:Johnson Brothers ...
3,Greenworkz USA Inc,Management Scientific and Technical Consulti...,Greenworkz USA Inc:Welcome to Greenworkz USA I...
4,T-Rex Arizona Baseball CLB LLC,Other Amusement and Recreation Industries,T-Rex Arizona Baseball CLB LLC:Under the leade...


In [ ]:
# train_df['combo'] = train_df['Sector_Description'] + '##' + train_df['Subsector_Description']
# dic = dict()
# for i in range(train_df.shape[0]):
#     if train_df.iloc[i, 0] not in dic:
#         dic[train_df.iloc[i, 0]] = list()
#         dic[train_df.iloc[i,0]].append(train_df.iloc[i,3])
#     else:
#         if train_df.iloc[i,3] not in dic[train_df.iloc[i,0]]:
#             dic[train_df.iloc[i,0]].append(train_df.iloc[i,3])

In [ ]:
# label_text_content = list()
# for k, v in dic.items():
#     label_text_content.append(k)
#     for i in v:
#         label_text_content.append(i)

In [ ]:
label_text_content = list()
for i in range(train_df.shape[0]):
    if train_df.iloc[i,1] not in label_text_content:
        label_text_content.append(train_df.iloc[i,1])

In [ ]:
train_df.iloc[0,1]

'Personal Care Services'

In [ ]:
test_df = train_df.iloc[:100,:]
val_df = train_df.iloc[100:1100, :]
train_df = train_df.iloc[1100:train_df.shape[0], :]

In [ ]:
len(val_df)

1000

In [ ]:
train_text_content = list()
val_text_content = list()
for i in range(train_df.shape[0]):
    text_1 = train_df.iloc[i,2] + '^' + train_df.iloc[i, 1]
    train_text_content.append(text_1)
for i in range(val_df.shape[0]):    
    text_2 = val_df.iloc[i,2] + '^' + val_df.iloc[i, 1]
    val_text_content.append(text_2)


In [ ]:
test_text_content = list()
for i in range(test_df.shape[0]):
    text = test_df.iloc[i,2]
    test_text_content.append(text)

In [33]:
with open('data/train.txt', 'w') as file:
    for line in train_text_content:
        file.write(line + '\n') 
with open('data/dev.txt', 'w') as file:
    for line in val_text_content:
        file.write(line + '\n') 
with open('data/data.txt', 'w') as file:
    for line in test_text_content:
        file.write(line + '\n') 
with open('data/label.txt', 'w') as file:
    for line in label_text_content:
        file.write(line + '\n') 

In [ ]:
# with open('dataset/train.txt', 'r', encoding='utf-8') as f:
#     for line in f:
#         items = line.strip().split('^')
#         print(items)
#         break

# Data Augmentation

In [ ]:
!python PaddleNLP/applications/text_classification/multi_class/analysis/aug.py \
    --create_n 2 \
    --aug_percent 0.1 \
    --aug_strategy mix \
    --aug_type mlm \
    --train_path "data/train.txt" \
    --aug_path "data/aug.txt"

[2023-12-07 23:57:59,497] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.modeling.ErnieForMaskedLM'> to load 'ernie-1.0-large-zh-cw'.
[2023-12-07 23:57:59,498] [    INFO] - Model config ErnieConfig {
  "attention_probs_dropout_prob": 0.1,
  "enable_recompute": false,
  "fuse": false,
  "hidden_act": "relu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "ernie",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "paddlenlp_version": null,
  "pool_act": "tanh",
  "task_id": 0,
  "task_type_vocab_size": 3,
  "type_vocab_size": 2,
  "use_task_id": true,
  "vocab_size": 18000
}

[2023-12-07 23:57:59,499] [    INFO] - Configuration saved in /home/aistudio/.paddlenlp/models/ernie-1.0-large-zh-cw/config.json
[2023-12-07 23:57:59,500] [    INFO] - Downloading ernie_1.0_large_zh_cw.pdparams from https://bj.bcebos

In [34]:
# Combine Augmented data and training data together
!cat data/aug.txt data/train.txt > data/train_aug.txt

# Model Training

In [36]:
! python PaddleNLP/applications/text_classification/multi_class/train.py \
    --do_train \
    --do_eval \
    --do_export \
    --model_name_or_path ernie-2.0-base-en \
    --output_dir checkpoint \
    --device gpu \
    --num_train_epochs 100 \
    --early_stopping True \
    --early_stopping_patience 5 \
    --learning_rate 3e-5 \
    --max_length 128 \
    --train_path ./data/train_aug.txt \
    --per_device_eval_batch_size 32 \
    --per_device_train_batch_size 32 \
    --metric_for_best_model accuracy \
    --load_best_model_at_end \
    --logging_steps 5 \
    --evaluation_strategy epoch \
    --save_strategy epoch \
    --save_total_limit 1

# Model Evaluation

In [37]:
! python PaddleNLP/applications/text_classification/multi_class/train.py \
    --do_eval \
    --debug True \
    --device gpu \
    --model_name_or_path checkpoint \
    --output_dir checkpoint \
    --per_device_eval_batch_size 32 \
    --max_length 128 \
    --test_path './data/dev.txt'

[2023-12-08 00:37:25,240] [ WARNING] - evaluation_strategy reset to IntervalStrategy.STEPS for do_eval is True. you can also set evaluation_strategy='epoch'.
[2023-12-08 00:37:25,240] [    INFO] - using `logging_steps` to initialize `eval_steps` to 100
[2023-12-08 00:37:25,240] [    INFO] - The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
[2023-12-08 00:37:25,242] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.modeling.ErnieForSequenceClassification'> to load 'checkpoint'.
[2023-12-08 00:37:25,242] [    INFO] - loading configuration file checkpoint/config.json
[2023-12-08 00:37:25,244] [    INFO] - Model config ErnieConfig {
  "architectures": [
    "ErnieForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "dtype": "float32",
  "enabl

# Model Prediction

In [38]:
from paddlenlp import Taskflow

cls = Taskflow("text_classification", task_path='checkpoint/export', is_static_model=True)
cls(test_df.iloc[0, 2])

[2023-12-08 00:37:35,145] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load 'checkpoint/export'.
[2023-12-08 00:37:37,492] [    INFO] - Load id2label from checkpoint/export/id2label.json.


[{'predictions': [{'label': 'Personal Care Services',
    'score': 0.9765003992179774}],
  'text': 'Mariposa Hair Salon:When you choose Mariposa at Jason Avenue, youre choosing a life of style and sophistication. Youll enjoy top-of-the-line in-home features to make your everyday routine easy, from high-speed internet and cable TV access to a generous porch or balcony with a storage closet. Once you step outside, youll be greeted by a host of community amenities perfect for active 55+ adults, from a dog park to a petanque court and horseshoe pit. Make every minute count at Mariposa at Jason Avenue. The on'}]